# Project 2: Ames Housing Data and Kaggle Challenge

--- 
# Kaggle Submission
---
- 1. cleaning the test.csv dataset using the same process I used for the training.csv
- 2. use the chosen model to predict sale price with the test.csv dataset

*All libraries used should be added here*

In [1]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import make_column_selector
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn import metrics
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import RFE
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import warnings
warnings.filterwarnings('ignore')
import os

### Data Import and Cleaning test.csv Dataset using the same process I used to clean training.csv

In [2]:
# read in the data
redotestcsv = pd.read_csv('datasets/test_kaggledownload.csv')
redotestcsv.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,893,20,RL,70.0,8414,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2006,WD,Normal
1,1106,60,RL,98.0,12256,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
2,414,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,3,2010,WD,Normal
3,523,50,RM,50.0,5000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,10,2006,WD,Normal
4,1037,20,RL,89.0,12898,Pave,NaN,IR1,HLS,AllPub,...,0,0,NaN,NaN,NaN,0,9,2009,WD,Normal


In [3]:
redotestcsv.shape

(365, 80)

In [4]:
redotestcsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             365 non-null    int64  
 1   MSSubClass     365 non-null    int64  
 2   MSZoning       365 non-null    object 
 3   LotFrontage    306 non-null    float64
 4   LotArea        365 non-null    int64  
 5   Street         365 non-null    object 
 6   Alley          21 non-null     object 
 7   LotShape       365 non-null    object 
 8   LandContour    365 non-null    object 
 9   Utilities      365 non-null    object 
 10  LotConfig      365 non-null    object 
 11  LandSlope      365 non-null    object 
 12  Neighborhood   365 non-null    object 
 13  Condition1     365 non-null    object 
 14  Condition2     365 non-null    object 
 15  BldgType       365 non-null    object 
 16  HouseStyle     365 non-null    object 
 17  OverallQual    365 non-null    int64  
 18  OverallCon

Exploratory

### Checking for nulls

In [5]:
#return list of columns with nulls
#housing.columns[housing.isnull().any()]
nulls = pd.DataFrame(redotestcsv.isnull().sum())
nulls.loc[nulls[0] != 0]

#19 cols with nulls

,0
LotFrontage,59
Alley,344
MasVnrType,4
MasVnrArea,4
BsmtQual,10
BsmtCond,10
BsmtExposure,11
BsmtFinType1,10
BsmtFinType2,11
FireplaceQu,178


### Dropping rows & columns where it's helpful

In [6]:
#dropping rows in columns where there aren't that many nulls

redotestcsv.dropna(subset=['MasVnrType','MasVnrArea','Electrical'], inplace=True)

### Engineer new columns
looking at nulls for some of these values, I decided to create new columns that might be more helpful
- does it have a garage?
- does it have a pool?
- does it have a basement?
- does it have a fence?
- does it have an alley?

In [7]:
redotestcsv['has_garage'] = np.where(redotestcsv['GarageType'].isna(), 0, 1)
redotestcsv['has_pool'] = np.where(redotestcsv['PoolQC'].isna(), 0, 1)
redotestcsv['has_basement'] = np.where(redotestcsv['BsmtQual'].isna(), 0, 1)
redotestcsv['has_fence'] = np.where(redotestcsv['Fence'].isna(), 0, 1)
redotestcsv['has_alley'] = np.where(redotestcsv['Alley'].isna(), 0, 1)

In [8]:
#doublechecking that columns were mapped correctly
redotestcsv['has_alley'].value_counts()

0    340
1     21
Name: has_alley, dtype: int64

In [9]:
redotestcsv['Alley'].value_counts()

Pave    15
Grvl     6
Name: Alley, dtype: int64

### Drop columns that have a very high percentage of nulls (>than 50% nulls). The values in the newly engineered columns will be more helpful

In [10]:
#drop columns that have a very high percentage of nulls (>than 50% nulls). The values in the newly engineered columns will be more helpful

redotestcsv.drop(columns = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence'], inplace=True)

In [11]:
redotestcsv.shape

(361, 81)

### Rename columns

In [12]:
# remove spaces in column names
redotestcsv.columns = [col.replace(' ', '') for col in redotestcsv.columns]

In [13]:
# remove spaces in column names
redotestcsv.columns = [col.replace('/', '_') for col in redotestcsv.columns]

In [14]:
# rename columns to lowercase
redotestcsv.columns = [col.lower() for col in redotestcsv.columns]     #list comprehension to rename cols
redotestcsv.head(2)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,lotshape,landcontour,utilities,lotconfig,...,miscval,mosold,yrsold,saletype,salecondition,has_garage,has_pool,has_basement,has_fence,has_alley
0,893,20,RL,70.0,8414,Pave,Reg,Lvl,AllPub,Inside,...,0,2,2006,WD,Normal,1,0,1,1,0
1,1106,60,RL,98.0,12256,Pave,IR1,Lvl,AllPub,Corner,...,0,4,2010,WD,Normal,1,0,1,0,0


# 3. Pre-Processing

In [15]:
nulls = pd.DataFrame(redotestcsv.isnull().sum())
nulls.loc[nulls[0] != 0]

,0
lotfrontage,58
bsmtqual,10
bsmtcond,10
bsmtexposure,11
bsmtfintype1,10
bsmtfintype2,11
garagetype,23
garageyrblt,23
garagefinish,23
garagequal,23


#### - create imputers to address nulls for nominal, ordinal, discrete, and continuous variables

In [16]:
#impute with median
imputer1 = SimpleImputer(strategy = 'median')

# impute with str 'NA'
imputer2 = SimpleImputer(strategy = 'constant', fill_value='NA')

# # imputer with most frequent
# imputer3 = SimpleImputer(strategy = 'most_frequent')

# making a column transformer
imputers = make_column_transformer(
                (imputer1, ['lotfrontage', 'garageyrblt']),
                (imputer2, ['bsmtqual', 'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2', 'garagetype', 'garagefinish', 'garagequal', 'garagecond', 'miscfeature']),
                # (imputer3, ['masvnrtype']),
                remainder = 'passthrough',
                verbose_feature_names_out=False)  #we need false so that the feature names come out in the form we want

In [17]:
# use the pipe above to impute the nulls
# turn the output into a dataframe with the column names
redotestcsv_imp = pd.DataFrame(imputers.fit_transform(redotestcsv), columns=imputers.get_feature_names_out())

In [18]:
nulls = pd.DataFrame(redotestcsv_imp.isna().sum())
nulls.loc[nulls[0] != 0]

,0


In [19]:
redotestcsv_imp['lotfrontage'].value_counts()

# this matches what I expected. median is 68 and 160 (nulls) + 16

67.0     61
60.0     38
80.0     18
70.0     16
75.0     14
         ..
140.0     1
120.0     1
89.0      1
56.0      1
105.0     1
Name: lotfrontage, Length: 75, dtype: int64

#### - create ordinal feature encoder pipeline

In [20]:
# cols I dropped & don't need to ordinal impute - 'Pool QC', 'Fireplace Qu', 'Fence'


# 'Lot Shape'
lotshape=['IR3','IR2', 'IR1','Reg'] 
ord1 = OrdinalEncoder(categories=[lotshape],dtype=int)

# Utilities
util=['ELO','NoSeWa','NoSewr','AllPub']
ord2 = OrdinalEncoder(categories=[util],dtype=int)

# Land Slope
lslope=['Sev', 'Mod','Gtl']
ord3= OrdinalEncoder(categories=[lslope],dtype=int)


# # Overall Qual, Overall Cond - talked to ben and he said it might be unnecessary to encode these b/c they are already in valued order
# overall=['1', '2', '3','4','5','6','7','8','9','10']
# ord4= OrdinalEncoder(categories=[overall],dtype=int)


# Exter Qual, Exter Cond, Bsmt Cond, Heating QC, Kitchen Qual, Garage Qual, Garage Cond, Bsmt Qual
# don't include b/c dropped: Pool QC, Fireplace Qu
exter=['NA','Po','Fa','TA','Gd','Ex']
ord5= OrdinalEncoder(categories=[exter],dtype=int)


# Bsmt Exposure
bsmtexp=['NA','No','Mn','Av','Gd']
ord6= OrdinalEncoder(categories=[bsmtexp],dtype=int)


# BsmtFin Type 1, BsmtFinType 2
bsmtfin=['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
ord7= OrdinalEncoder(categories=[bsmtfin],dtype=int)


# Electrical
elec=['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix']
ord8= OrdinalEncoder(categories = [elec],dtype=int)


#Functional
func=['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1','Typ']
ord9= OrdinalEncoder(categories=[func],dtype=int)


#Garage Finish
gar=['NA','Unf','RFn','Fin']
ord10= OrdinalEncoder(categories=[gar],dtype=int)


#paved driveway
drive=['N','P','Y']
ord11= OrdinalEncoder(categories=[drive],dtype=int)


# Fence - I don't need this since I dropped fence col
# fence=['NA','MnWw','GdWo','MnPrv','GdPrv']
# ord12= OrdinalEncoder(categories=[[fence]])


# combine all imputers into one imputer pipe
ord_imputers = make_column_transformer(
                (ord1, ['lotshape']),
                (ord2, ['utilities']),
                (ord3, ['landslope']),
                (ord5, ['exterqual']),
                (ord5, ['extercond']),
                (ord5, ['bsmtcond']),
                (ord5, ['heatingqc']),
                (ord5, ['kitchenqual']),
                (ord5, ['garagequal']),
                (ord5, ['garagecond']),
                (ord5, ['bsmtqual']),
                (ord6, ['bsmtexposure']),
                (ord7, ['bsmtfintype1']),
                (ord7, ['bsmtfintype2']),
                (ord8, ['electrical']),
                (ord9, ['functional']),
                (ord10, ['garagefinish']),
                (ord11, ['paveddrive']),
                remainder = 'passthrough',
                verbose_feature_names_out=False)

# what I tried to do with ord5 - (ord5, ['exterqual','extercond', 'bsmtcond', 'heatingqc', 'kitchenqual', 'garagequal','garagecond', 'bsmtqual']),

In [21]:
ord_imputers.fit_transform(redotestcsv_imp)

array([[3, 3, 2, ..., 1, 1, 0],
       [2, 3, 2, ..., 1, 0, 0],
       [3, 3, 2, ..., 1, 0, 1],
       ...,
       [3, 3, 2, ..., 1, 0, 0],
       [1, 3, 2, ..., 1, 0, 0],
       [3, 3, 2, ..., 1, 0, 0]], dtype=object)

In [22]:
# apply the ord_imputer, convert to dataframe and assign to new object
redotestcsv_imp2 = pd.DataFrame(ord_imputers.fit_transform(redotestcsv_imp), columns = ord_imputers.get_feature_names_out())

### double-checking that features that I ordinal encoded do match up to the category list

In [23]:
redotestcsv_imp['extercond'].value_counts()

TA    318
Gd     38
Fa      4
Ex      1
Name: extercond, dtype: int64

In [24]:
redotestcsv_imp2['extercond'].value_counts()

3    318
4     38
2      4
5      1
Name: extercond, dtype: int64

In [25]:
#everything was converted to object. changing necessary cols to int/float
cols_to_change = ['id','lotfrontage', 'lotarea', 'overallqual',
       'overallcond', 'yearbuilt', 'masvnrarea', 'bsmtfinsf1',
       'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', '1stflrsf', '2ndflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'totrmsabvgrd',
       'fireplaces', 'garageyrblt', 'garagecars', 'garagearea', 'wooddecksf',
       'openporchsf', 'enclosedporch', '3ssnporch', 'screenporch', 'poolarea',
       'miscval', 'mosold', 'yrsold', 'has_garage', 'has_pool', 'has_basement',
       'has_fence', 'has_alley','lotshape','utilities','landslope','exterqual','extercond','bsmtcond',
       'heatingqc', 'kitchenqual', 'garagequal', 'garagecond','bsmtqual','bsmtexposure','bsmtfintype1',
        'bsmtfintype2', 'electrical','functional','garagefinish','paveddrive','overallqual','overallcond']


for col in cols_to_change:
    redotestcsv_imp2[col] = pd.to_numeric(redotestcsv_imp2[col])
    
#deleted: pid, yrremod

In [26]:
#checking to make sure these are the only list of categoricals
redotestcsv_imp2.select_dtypes(object).columns

Index(['garagetype', 'miscfeature', 'mssubclass', 'mszoning', 'street',
       'landcontour', 'lotconfig', 'neighborhood', 'condition1', 'condition2',
       'bldgtype', 'housestyle', 'yearremodadd', 'roofstyle', 'roofmatl',
       'exterior1st', 'exterior2nd', 'masvnrtype', 'foundation', 'heating',
       'centralair', 'saletype', 'salecondition'],
      dtype='object')

### Dummy categorical features on test.csv

In [27]:
# dummy all the cat columns - drop first
# dummy_na will separate out a column for non-value
redotestcsv_dummied = pd.get_dummies(data=redotestcsv_imp2, drop_first=True)

In [28]:
redotestcsv_dummied.shape

(361, 248)

In [29]:
redotestcsv_imp2.shape

(361, 81)

In [30]:
redotestcsv_dummied.head()

,lotshape,utilities,landslope,exterqual,extercond,bsmtcond,heatingqc,kitchenqual,garagequal,garagecond,...,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial
0,3,3,2,3,3,3,3,3,3,3,...,0,0,0,0,0,1,0,0,1,0
1,2,3,2,4,3,3,5,4,3,3,...,0,0,0,0,0,1,0,0,1,0
2,3,3,2,3,3,3,4,3,3,3,...,0,0,0,0,0,1,0,0,1,0
3,3,3,2,3,4,3,5,3,3,3,...,0,0,0,0,0,1,0,0,1,0
4,2,3,2,4,3,3,5,5,3,3,...,0,0,0,0,0,1,0,0,1,0


Before scaling, dropping 'id' and 'pid'

In [31]:
#testcsv_dummied
redotestcsv_toscale = redotestcsv_dummied.drop(columns = ['id'])
redotestcsv_id = redotestcsv_dummied[['id']]

In [32]:
redotestcsv_toscale

,lotshape,utilities,landslope,exterqual,extercond,bsmtcond,heatingqc,kitchenqual,garagequal,garagecond,...,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial
0,3,3,2,3,3,3,3,3,3,3,...,0,0,0,0,0,1,0,0,1,0
1,2,3,2,4,3,3,5,4,3,3,...,0,0,0,0,0,1,0,0,1,0
2,3,3,2,3,3,3,4,3,3,3,...,0,0,0,0,0,1,0,0,1,0
3,3,3,2,3,4,3,5,3,3,3,...,0,0,0,0,0,1,0,0,1,0
4,2,3,2,4,3,3,5,5,3,3,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2,3,2,3,3,3,3,4,3,3,...,0,0,0,0,0,1,0,0,1,0
357,3,3,2,3,3,3,3,3,3,3,...,0,0,0,0,0,1,0,0,1,0
358,3,3,2,4,3,3,5,4,3,3,...,0,0,0,0,0,1,0,0,1,0
359,1,3,2,4,3,3,5,4,3,3,...,0,0,0,0,0,1,0,0,1,0


### Use standard scalar to fit and transform data

In [33]:
#instantiate the StandardScaler
sc = StandardScaler()

In [34]:
#transform
Z_test = pd.DataFrame(sc.fit_transform(redotestcsv_toscale), columns = redotestcsv_toscale.columns)

In [35]:
# join the ids and scaled df back together
Z_test = pd.concat([redotestcsv_id, Z_test], axis=1)

In [36]:
Z_test.tail()

,id,lotshape,utilities,landslope,exterqual,extercond,bsmtcond,heatingqc,kitchenqual,garagequal,...,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial
356,989,-1.174330,0.0,0.222745,-0.697735,-0.290948,0.134892,-1.218972,0.724675,0.304752,...,-0.074639,-0.074639,-0.052705,-0.306495,-0.074639,0.405606,-0.130005,-0.091542,0.499134,-0.31719
357,244,0.660878,0.0,0.222745,-0.697735,-0.290948,0.134892,-1.218972,-0.796300,0.304752,...,-0.074639,-0.074639,-0.052705,-0.306495,-0.074639,0.405606,-0.130005,-0.091542,0.499134,-0.31719
358,1343,0.660878,0.0,0.222745,1.063680,-0.290948,0.134892,0.896648,0.724675,0.304752,...,-0.074639,-0.074639,-0.052705,-0.306495,-0.074639,0.405606,-0.130005,-0.091542,0.499134,-0.31719
359,1058,-3.009538,0.0,0.222745,1.063680,-0.290948,0.134892,0.896648,0.724675,0.304752,...,-0.074639,-0.074639,-0.052705,-0.306495,-0.074639,0.405606,-0.130005,-0.091542,0.499134,-0.31719
360,1419,0.660878,0.0,0.222745,-0.697735,-0.290948,0.134892,-1.218972,-0.796300,0.304752,...,-0.074639,-0.074639,-0.052705,-0.306495,-0.074639,-2.465447,-0.130005,-0.091542,0.499134,-0.31719


In [37]:
#keep the columns that match the training final df set used for modeling
Z_test_final = Z_test[['id','overallqual', 'exterqual', 'kitchenqual', 'garagearea', 'garagefinish', 'fullbath', 'masvnrarea', 'garageyrblt',
       'totrmsabvgrd', 'heatingqc', 'neighborhood_NridgHt', 'fireplaces',
       'bsmtfinsf1', 'bsmtexposure', 'saletype_New', 'openporchsf',
       'bsmtfintype1', 'exterior2nd_VinylSd', 'lotfrontage',
       'masvnrtype_Stone', 'wooddecksf', 'halfbath', 'lotarea', 'paveddrive',
       'neighborhood_StoneBr', 'roofstyle_Hip', 'bsmtfullbath', 'centralair_Y',
       'garagetype_BuiltIn', 'masvnrtype_BrkFace', 'neighborhood_NoRidge',
       'bsmtcond', 'housestyle_2Story', 'landcontour_HLS', 'has_garage',
       'bsmtunfsf', 'exterior2nd_CmentBd', 'screenporch', 'bedroomabvgr',
       'lotconfig_CulDSac', 'neighborhood_Somerst', 'functional',
       'neighborhood_Timber', 'condition1_Norm', 'condition2_PosA',
       'neighborhood_MeadowV', 'overallcond', 'neighborhood_BrDale',
       'kitchenabvgr', 'condition1_Feedr', 'bldgtype_Twnhs', 'has_alley',
       'exterior2nd_MetalSd', 'neighborhood_BrkSide', 'foundation_Slab',
       'exterior2nd_HdBoard', 'neighborhood_Sawyer', 'enclosedporch',
       'has_fence', 'neighborhood_NAmes', 'exterior2nd_Wd Sdng',
       'neighborhood_IDOTRR', 'neighborhood_OldTown', 'neighborhood_Edwards',
       'exterior1st_Wd Sdng', 'garagetype_NA', 'electrical',
       'mszoning_RM', 'lotshape', 'foundation_CBlock', 'garagetype_Detchd']]

#pid, yearremod_add, saletype_WD

In [38]:
pd.set_option('display.max_rows', 200)

In [39]:
Z_test_final.shape

(361, 72)

### datasets for modeling
- df_model_train
- df_model_test
- y_train
- y_test

In [40]:
#read in datasets that I used for modelling

housing = pd.read_csv('cleaned_datasets/housing_preprocessed1_data.csv')
df_model_train = pd.read_csv('cleaned_datasets/housing_df_model_train.csv')
df_model_test = pd.read_csv('cleaned_datasets/housing_df_model_test.csv')
y_train = pd.read_csv('cleaned_datasets/housing_y-train.csv')
y_test = pd.read_csv('cleaned_datasets/housing_y-test_data.csv')

In [41]:
df_model_train.drop(columns=['saletype_WD ', 'yearremod_add'], inplace=True)
df_model_test.drop(columns=['saletype_WD ', 'yearremod_add'], inplace=True)

In [42]:
df_model_train_final = df_model_train.drop(columns=['pid', 'id'])
df_model_test_final = df_model_test.drop(columns=['pid', 'id'])
y_train = y_train.drop(columns=['id'])
y_test = y_test.drop(columns=['id'])

In [43]:
df_model_train_final.shape

(1370, 71)

In [44]:
df_model_test_final.shape

(676, 71)

In [45]:
Z_test_final.shape

(361, 72)

In [46]:
# baseline
y_train.mean()[0]

181205.50218978102

#### Model 1 - Linear Regression

In [47]:
#instantiate and fit OLS model
lr = LinearRegression()
lr.fit(df_model_train_final, y_train)

LinearRegression()

In [48]:
# r2 for test set from train.csv
lr.score(df_model_train_final, y_train)

0.9036253362877437

In [49]:
#calculate RMSE train
y_pred = lr.predict(df_model_train_final)
mean_squared_error(y_train, y_pred, squared=False)

24725.056444873153

In [50]:
#calculate RMSE test
y_test_pred = lr.predict(df_model_test_final)
mean_squared_error(y_test, y_test_pred, squared=False)

26756.19818440426

In [51]:
#checking the model on the test data
lr.score(df_model_test_final, y_test)

0.8838016793174307

In [52]:
lr.coef_

array([[ 1.65310137e+04,  4.94681080e+03,  6.71453072e+03,
         6.13156120e+03,  1.16738843e+03,  7.05071777e+03,
         7.62191473e+03, -4.49280019e+02,  1.26006131e+04,
         1.41924082e+03,  6.97075593e+03,  3.98518771e+03,
         1.61386874e+04,  5.31062174e+03,  4.36582765e+03,
         2.73885193e+03, -5.83392255e+01, -2.23297065e+02,
         4.02579398e+03, -6.03187241e+02,  1.86727274e+03,
         4.50894426e+03,  5.96449875e+03,  1.56137122e+03,
         6.18597318e+03,  3.11817745e+03,  3.32365602e+03,
        -1.15239616e+02,  2.46305419e+03, -4.64935910e+03,
         5.34471238e+03, -3.01464211e+03,  1.09295009e+02,
         2.66587015e+03, -6.49273163e+16,  7.90635122e+03,
         2.08836098e+03,  4.72159486e+03, -1.62622036e+03,
         2.00748056e+03,  1.06459467e+03,  1.50004827e+03,
        -9.64740048e+02,  2.37071533e+03,  1.64793861e+03,
        -7.29049461e+02,  5.25937033e+03,  4.41269857e+02,
        -4.45993714e+03,  4.49369592e+02, -1.57590550e+0

In [53]:
pd.DataFrame(lr.coef_[0], df_model_train_final.columns).sort_values(by=0, ascending=False)

,0
overallqual,1.653101e+04
bsmtfinsf1,1.613869e+04
totrmsabvgrd,1.260061e+04
bsmtunfsf,7.906351e+03
masvnrarea,7.621915e+03
fullbath,7.050718e+03
neighborhood_NridgHt,6.970756e+03
kitchenqual,6.714531e+03
neighborhood_StoneBr,6.185973e+03
garagearea,6.131561e+03


In [54]:
print('baseline:', y_train.mean())

print("-----LR/OLS----- ")
print('rmse train:', mean_squared_error(y_train, y_pred, squared=False))
print('rmse test:', mean_squared_error(y_test, y_test_pred, squared=False))
print('r2 train:', lr.score(df_model_train_final, y_train))
print('r2 test:', lr.score(df_model_test_final, y_test))

baseline: saleprice    181205.50219
dtype: float64
-----LR/OLS----- 
rmse train: 24725.056444873153
rmse test: 26756.19818440426
r2 train: 0.9036253362877437
r2 test: 0.8838016793174307


### Run test csv data through the model built above to predict sale price

In [55]:
Z_test_final2 = Z_test_final.drop(columns=['id'])

In [56]:
lr.predict(Z_test_final2)[:10]

array([[151966.56455877],
       [325509.81329573],
       [ 99876.14138226],
       [174580.24439782],
       [309627.81329573],
       [ 75007.84609542],
       [232946.94027293],
       [135686.56455877],
       [ 62303.84609542],
       [168035.81329573]])

In [57]:
pd.set_option('display.max_rows', 200)

In [58]:
kaggle_preds = pd.DataFrame(lr.predict(Z_test_final2))
kaggle_preds.rename(columns = {0: 'SalePrice'}, inplace=True)
kaggle_preds[:15]

,SalePrice
0,151966.564559
1,325509.813296
2,99876.141382
3,174580.244398
4,309627.813296
5,75007.846095
6,232946.940273
7,135686.564559
8,62303.846095
9,168035.813296


In [59]:
kaggle_preds.mean()

SalePrice    181222.226173
dtype: float64

In [61]:
kaggle = pd.concat([Z_test_final, kaggle_preds], axis=1)
kaggle = kaggle[['id', 'SalePrice']]
kaggle.rename(columns={'id': 'Id'}, inplace=True)
kaggle[:20]

,Id,SalePrice
0,893,151966.564559
1,1106,325509.813296
2,414,99876.141382
3,523,174580.244398
4,1037,309627.813296
5,615,75007.846095
6,219,232946.940273
7,1161,135686.564559
8,650,62303.846095
9,888,168035.813296


In [62]:
kaggle = kaggle.rename(columns={'id': 'Id', 'SalePrice': 'sample_soln'})[:260]
kaggle = kaggle.reset_index().drop(columns='Id')

In [63]:
kaggle = kaggle.rename(columns={'index':'Id'})
kaggle

,Id,sample_soln
0,0,151966.564559
1,1,325509.813296
2,2,99876.141382
3,3,174580.244398
4,4,309627.813296
...,...,...
255,255,101064.244398
256,256,125532.244398
257,257,185761.437582
258,258,261087.437582


In [64]:
#export predictions to csv
kaggle.to_csv('DS_kaggle_submission_redo.csv', index = False)

#commenting out so it doesn't reexport